# Proyecto Final: Resúmenes de Reseñas Académicas


In [1]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2c9022e3edebb11ce370325745631ee1708577ddc61911ccd3d70f9c979dafb3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
import pandas as pd
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

comentarios = pd.read_csv('CSV/Comentarios_Profesores_Generalizado.csv')
listado = pd.read_csv('CSV/Listado_profesores_Generalizado.csv')
comentarios_referencia = pd.read_csv('CSV/Comentarios_Referencia.csv')

try:
    modelo_resumen = pipeline("summarization", model="facebook/bart-large-cnn")
    print("Modelo cargado exitosamente.")
except Exception as e:
    print("Error cargando el modelo:", e)
    #modelo_beto = None
    modelo_resumen = None

def listado_profesores():
    print("Listado de profesores:")
    #Obtenemos el listado
    profesores_unicos = comentarios['Profesor'].unique()
    for index, profesor in enumerate(profesores_unicos):
        print(f"\t{index + 1}. {profesor}")

def ver_comentarios(profesores_unicos):
    try:
        numero_profesor = int(input('Ingrese el numero del profesor: '))
        if 1 <= numero_profesor <= len(profesores_unicos):
            profesor = profesores_unicos[numero_profesor - 1]
            comentarios_profesor = comentarios[comentarios['Profesor'] == profesor]
            if comentarios_profesor.empty:
                print(f"No hay comentarios disponibles para {profesor}.")
                return
            print(f"Comentarios del profesor {profesor}:")
            for index, comentario in comentarios_profesor.iterrows():
                print(f"\t{index+1}. {comentario['Comentario']}")
        else:
            print('Opcion invalida, intente de nuevo')
    except ValueError:
        print('Opcion invalida, intente de nuevo')

def generar_opinion(profesores_unicos):
    try:
        numero_profesor = int(input('Ingrese el numero del profesor: '))
        if 1 <= numero_profesor <= len(profesores_unicos):
            profesor = profesores_unicos[numero_profesor - 1]
            print(f"Generando opinión del profesor {profesor}:")
            comentarios_profesor = comentarios[comentarios['Profesor'] == profesor]["Comentario"]
            if comentarios_profesor.empty:
                print(f"No hay comentarios disponibles para {profesor}.")
                return

            comentarios_profesor = comentarios_profesor.dropna()
            comentarios_profesor = comentarios_profesor.astype(str)

            if len(comentarios_profesor) <= 4:
                print(f"El profesor {profesor} tiene {len(comentarios_profesor)} comentarios. Mostrando todos los comentarios disponibles:")
                for comentario in comentarios_profesor:
                    print(f"- {comentario}")
                return

            elif 5 <= len(comentarios_profesor) < 9:
                texto_base = ' '.join(comentarios_profesor.tolist())
            else:
                texto_base = ' '.join(comentarios_profesor.sample(n=9, random_state=37).tolist())

            print(f"Texto base para generación: {texto_base}")

            if modelo_resumen is None:
                print("El modelo de resumen no está disponible. Verifica su carga.")
                return

            if not texto_base.strip():
                print("No hay suficiente texto base para generar un resumen.")
                return

            try:
                opinion_generada = modelo_resumen(
                    texto_base, max_length=300, min_length=150, do_sample=False
                )[0]['summary_text']
                print("\n############################################\n")
                print(f"Resumen generado: {opinion_generada}")

                return opinion_generada #Obtener la opinion

            except Exception as e:
                print("Error generando la opinión:", e)
                return None
        else:
            print('Opción inválida, intente de nuevo 1')
            return None
    except ValueError:
        print('Opción inválida, intente de nuevo 2')
        return None

# Calcular la métrica de BLEU
def calcular_bleu(opinion_generada, comentarios_ref):
    # Tokenizar los comentarios de referencia y la opinión generada
    referencia_bleu = [ref.split() for ref in comentarios_ref]
    opinion_bleu = opinion_generada.split()

    #BLEU normal
    score_bleu = sentence_bleu(referencia_bleu, opinion_bleu)
    print(f"BLEU score: {score_bleu:.4f}")

    # Configuración para BLEU-1 y suavizado
    score_bleu1 = sentence_bleu(
        referencia_bleu,
        opinion_bleu,
        weights=(1, 0, 0, 0),  # Solo considera unigramas
        smoothing_function=SmoothingFunction().method1  # Aplica suavizado
    )
    print(f"BLEU-1 score: {score_bleu1:.4f}")
    return score_bleu, score_bleu1

# Calcular la métrica de ROUGE
def calcular_rouge(opinion_generada, comentarios_ref):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores_rouge = scorer.score(' '.join(comentarios_ref), opinion_generada)
    print("ROUGE scores:")
    print(f"ROUGE-1: {scores_rouge['rouge1']}") # Unigramas
    print(f"ROUGE-2: {scores_rouge['rouge2']}") # Bigramas
    print(f"ROUGE-L: {scores_rouge['rougeL']}") # Subsecuencia común más larga
    return scores_rouge

def evaluar_opinion(opinion_generada):
    numero_profesor = int(input('Ingrese el numero del profesor nuevamente: '))
    numeros_unicos = comentarios_referencia['Numero'].unique().astype(int)
    if numero_profesor in numeros_unicos:
        comentarios_ref = comentarios_referencia[comentarios_referencia['Numero'].astype(str) == str(numero_profesor)]["Comentario"].dropna() # Sin valores nulos
        print("\n ** Evaluando la opinión generada: **")
        for i, comentario in enumerate(comentarios_ref, 1):
            print(f"Opinión de referencia: {comentario}")
        # Evaluar con BLEU
        print("\nMétrica BLEU:")
        bleu_score = calcular_bleu(opinion_generada, comentarios_ref)
        # Evaluar con ROUGE
        print("\nMétrica ROUGE:")
        rouge_scores = calcular_rouge(opinion_generada, comentarios_ref)
        #return bleu_score, rouge_scores
    else:
        print('No existen comentarios de referencia para ese profesor')
        #return None, None

def sub_menu():
    profesores_unicos = comentarios['Profesor'].unique()
    while True:
        print('MENU')
        print('1. Ver comentarios')
        print('2. Generar opinion')
        print('3. Evaluar opinión generada')
        print('4. Salir')

        try:
            opcion = int(input('Ingrese una opcion: '))
            if opcion == 1:
                ver_comentarios(profesores_unicos)
            elif opcion == 2:
                opinion_generada = generar_opinion(profesores_unicos)
            elif opcion == 3:
                if opinion_generada:
                    #print(opinion_generada)
                    evaluar_opinion(opinion_generada)
                else:
                    print("Primero debe generar una opinión antes de evaluarla.")
            elif opcion == 4:
                break
            else:
                print('Opcion invalida, intente de nuevo')
        except ValueError:
            print('Opcion invalida, intente de nuevo')


if __name__ == '__main__':
    profesores_unicos = comentarios['Profesor'].unique()
    while True:
        print('MENU')
        print('1. Listado de profesores')
        print('2. Salir')

        try:
            opcion = int(input('Ingrese una opcion: '))
            if opcion == 1:
                listado_profesores()
                sub_menu()
            elif opcion == 2:
                break
            else:
                print('Opcion invalida, intente de nuevo')
        except ValueError:
            print('Opcion invalida, intente de nuevo')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...


 ** Evaluando la opinión generada: **
Opinión de referencia: Es mu engreído y egocéntrico pero a veces cuenta chascarrillos, sus clases son pura teoria y diapositivas, las practicas son individuales y puro copiar y pegar las sentencias para ejecutarlas, se tienen que entregar en la hora y media de clase y recomiendo llevar el software ya listo porque si no es imposible terminarlas, la asistencia vale un 80% y si tienes 3 faltas no te permite hacer examen, los examenes son difíciles y en dos partes pero si repasas las diapositivas te va bien, el proyecto final lo revisa durante los 3 parciales y es en equipo.

Métrica BLEU:
BLEU score: 0.0000
BLEU-1 score: 0.2242

Métrica ROUGE:
ROUGE scores:
ROUGE-1: Score(precision=0.27722772277227725, recall=0.27722772277227725, fmeasure=0.27722772277227725)
ROUGE-2: Score(precision=0.06, recall=0.06, fmeasure=0.06)
ROUGE-L: Score(precision=0.12871287128712872, recall=0.12871287128712872, fmeasure=0.12871287128712872)
MENU
1. Ver comentarios
2. Gene

Your max_length is set to 300, but your input_length is only 279. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=139)


Generando opinión del profesor Ana Karen Benitez:
Texto base para generación: No sabe jugar bien al billar :v se preocupa por que aprendas, hace muy dinamica y amena la clase, se explica bien Siento que para que un profesor sea bueno debe gustarle dar clase, es de los pocos profesores que le gusta dar clase y lo transmite, es fácil y dinámica su clase, (su sonrisa es otro asunto muy bonito :3) [Comentario esperando revisión] Super linda la profesora. No te sube nada de puntos para que pases y varios se enojan por eso, pero la recomiendo mucho. En las actividades de lectura y cosas así te ayuda en vez de sentarse, es buena! Switch Places, Taco Time, Maluma is Handsome.... :v :v :v :3 :3 :3 Auqnue está joven es bastante buena, explica bien la gramática y hace muchas actividades, te incita a participar porque es divertida y aparte está bonita :v la recomiendo

############################################

Resumen generado: Super linda la profesora. No sabe jugar bien al billar :v se preoc

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



 ** Evaluando la opinión generada: **
Opinión de referencia: Es una maestra joven y bastante buena, se nota que le gusta dar clase y la hace amena, por lo que es fácil, divertida y dinámica, te deja muchas actividades e incita a participar, pero tambien te ayuda, es bonita.

Métrica BLEU:
BLEU score: 0.0000
BLEU-1 score: 0.2400

Métrica ROUGE:
ROUGE scores:
ROUGE-1: Score(precision=0.28205128205128205, recall=0.5238095238095238, fmeasure=0.36666666666666664)
ROUGE-2: Score(precision=0.03896103896103896, recall=0.07317073170731707, fmeasure=0.05084745762711864)
ROUGE-L: Score(precision=0.14102564102564102, recall=0.2619047619047619, fmeasure=0.18333333333333332)
MENU
1. Ver comentarios
2. Generar opinion
3. Evaluar opinión generada
4. Salir
Ingrese una opcion: 4
MENU
1. Listado de profesores
2. Salir
Ingrese una opcion: 2
